In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
from advi.model import ADVIModel
from models.ard import Ard
from models.ard import sep_params
#from models.ard import joint_log_prob, return_initial_state, sep_params, log_likelihood, some_kind_of_loss
# NEXT TO DO MUST CHANGE INITAL CHAIN STATES TO SAMPLE FROM DISTRIBUTION
# Target distribution is proportional to: `exp(-x (1 + x))`.

# Initialize the HMC transition kernel.



/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def make_training_data(num_samples, dims, sigma):
  x = np.random.randn(dims, num_samples).astype(np.float64)
  w = sigma * np.random.randn(1, dims).astype(np.float64)
  noise = np.random.randn(num_samples).astype(np.float64)
  w[:,:int(dims/2)] = 0
  y = w.dot(x) + noise
    
  return y, x, w

def sep_training_test(y,x,test):
  y_train = y[:,test:]
  x_train = x[:,test:]
  
  y_test = y[:,:test]
  x_test = x[:,:test]
  return y_train, y_test, x_train, x_test
num_features = 100

y, x, w = make_training_data(200, num_features, 2)
print("y shape", y.shape)
print("x shape", x.shape)
print("w shape", w.shape)


('y shape', (1, 200))
('x shape', (100, 200))
('w shape', (1, 100))


In [3]:
y_train, y_test, x_train, x_test = sep_training_test(y,x,50)
print(y_train.shape)
print(y_test.shape)
print(x_train.shape)
print(x_test.shape)



(1, 150)
(1, 50)
(100, 150)
(100, 50)


In [4]:
model = Ard(100)
initial_chain_state = model.return_initial_state()
wstart, tau, alpha = sep_params(initial_chain_state, num_features)


In [5]:
def state_to_log_pred(states, y_test, x_test):
    sample_mean = tf.reduce_mean(states, axis=[0])
    return model.log_likelihood(y_test, x_test, states)

def state_to_loss(states, y_test, x_test):
    sample_mean = tf.reduce_mean(states, axis=[0])
    w, _, _ = sep_params(states, num_features)
    return model.some_kind_of_loss(y_test, x_test, w)

state_to_loss(initial_chain_state, y_test, x_test)

<tf.Tensor: shape=(), dtype=float64, numpy=896.4206956991478>

In [6]:
summary_writer = tf.compat.v2.summary.create_file_writer('/tmp/summary_chain', flush_millis=10000)
def trace_fn(state, results):
  with tf.compat.v2.summary.record_if(tf.equal(results.step % 10, 0)):
    tf.compat.v2.summary.scalar("loss", state_to_loss(state, y_test, x_test), step=tf.cast(results.step, tf.int64))
    tf.compat.v2.summary.scalar("log pred", state_to_log_pred(state, y_test, x_test), step=tf.cast(results.step, tf.int64))
    return ()

In [7]:
num_results = int(10000)
num_burnin_steps = int(800)
adaptive_hmc = tfp.mcmc.SimpleStepSizeAdaptation(
    tfp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=model.joint_log_prob,
        num_leapfrog_steps=3,
        step_size=0.1),
    num_adaptation_steps=int(num_burnin_steps * 0.8))

nuts = tfp.mcmc.NoUTurnSampler(
    target_log_prob_fn=model.joint_log_prob, step_size=1., max_tree_depth=10, max_energy_diff=1000.0,
    unrolled_leapfrog_steps=1, parallel_iterations=10, seed=None, name=None)

@tf.function
def run_chain():
  print("running chain")
  # Run the chain (with burn-in).
  states, is_accepted = tfp.mcmc.sample_chain(
      num_results=num_results,
      num_burnin_steps=num_burnin_steps,
      current_state=initial_chain_state,
      kernel=adaptive_hmc,
    trace_fn=trace_fn)
  return states, is_accepted

    

@tf.function
def run_chain_nuts():
  print("running chain")
  # Run the chain (with burn-in).
  states, is_accepted = tfp.mcmc.sample_chain(
      num_results=20,
      num_burnin_steps=num_burnin_steps,
      current_state=initial_chain_state,
      kernel=nuts)
  return states, is_accepted

In [8]:
@tf.function
def run_advi(nsteps, dim, log_prob, bijector,m):
    advi = ADVIModel(dim, log_prob, bijector,m)
    for t in range(nsteps):
        grad_mu, grad_omega = advi.gradients()
        advi.mu = tf.add(q.mu, step_size * grad_mu)
        advi.omega = tf.add(q.omega, step_size * grad_omega)
    # sample from eta distribution currently just once,
    # using current values of mu and sigma to return trained params theta
    theta = advi.sample()
    return theta

In [9]:
joint_log_prob2 = lambda *args: model.joint_log_prob(y_train, x_train, *args)
target = model.joint_log_prob
bij = tfp.bijectors.Log()
theta = run_advi(nsteps=40,
                 dim=model.num_params,
                 log_prob=joint_log_prob2,
                 bijector=bij,
                 m=10)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
('features', 100)
check1
check2
check3
('check4', <tf.Tensor 'add_2:0' shape=() dtype=float64>)
grad1 check
grad2 check
grad3 check
('res1', <tf.Tensor 'log_exp/forward/Exp:0' shape=(10, 201) dtype=float64>)
('zeta', <tf.Tensor 'Add:0' shape=(10, 201) dtype=float64>)
('grad1', None)


AttributeError: in converted code:

    <ipython-input-8-573de73d9857>:5 run_advi  *
        grad_mu, grad_omega = advi.gradients()
    advi/model.py:115 gradients  *
        assert grad1.shape == (nsamples, self._dim)

    AttributeError: 'NoneType' object has no attribute 'shape'


In [ ]:
with summary_writer.as_default():
  theta = run_advi(40, 100, log_prob, bij)
summary_writer.close()
start_loss = state_to_loss(initial_chain_state, y_test, x_test)
print("start loss", start_loss)
loss = state_to_loss(theta, y_test, x_test)
print("loss", loss)

print(is_accepted)


In [ ]:
initial_chain_state = return_initial_state(num_features)
sample_mean = tf.reduce_mean(states, axis=[0])
wend, alpha, tau = sep_params(sample_mean, num_features)

print("start", wstart)
loss_start = some_kind_of_loss(wstarty_test, x_test, )
print("loss start", loss_start)

print("actual",w)
print("end", wend)
print("start", wstart)
loss_end = some_kind_of_loss(wend,y_test, x_test, )
print("loss start", loss_start)
print("loss end", loss_end)


In [ ]:
sample_mean = tf.reduce_mean(states, axis=[0])
wend, alpha, tau = sep_params(sample_mean, num_features)

